In [116]:
from preprocess import dataset,data_deal
from sklearn.preprocessing import MinMaxScaler
from models import AE_MLP,get_models
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import KFold
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP
import pandas as pd
import joblib
from torch import optim

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [117]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
file = './data/data for predict.xlsx'
file2 = './data/total 12 data.xlsx'
data = pd.read_excel(file,engine="openpyxl")
data = np.array(data)
data2 = pd.read_excel(file2,engine="openpyxl")
data2 = np.array(data2)
l = len(data)
para_path = './save/parameter'
batch_size = 300
plot = True
np.set_printoptions(threshold=np.sys.maxsize)

In [118]:
print(data)

[[269.         114.         155.          13.6534       0.
    3.5          3.5         27.12758538  30.85207969 120.33844743
    3.94052045  84.08765142]
 [270.         114.         156.          13.5532       0.
    0.           0.          27.14436708  30.96591574 120.78584076
    3.94074074  84.13234452]
 [271.         114.         157.          13.4365       1.
   -5.5          4.5         27.16109706  31.10009911 121.31260287
    3.94095941  84.17689533]
 [272.         114.         158.          13.2473       0.
    0.           0.          27.17777568  31.32139998 122.1791989
    3.94117647  84.22130489]
 [273.         114.         159.          12.9951       5.
    0.5         -5.5         27.19440327  31.62387163 123.36245898
    3.94139194  84.26557417]
 [274.         114.         160.          12.828        0.
    0.           0.          27.21098019  31.82917454 124.16670002
    3.94160584  84.30970416]
 [275.         114.         161.          12.6436       2.
    1.5     

In [119]:
with open('config.json','r',encoding='utf-8')as f:
    config_list = json.load(f)
flag = 0

for config in config_list:
    Type = config['Type']
    N_size = config['N_size']
    lr = config['lr']
    k = config['k']
    alpha = config['alpha']
    input_size = config['input_size']
    epsilon = config['epsilon']
    seed1,seed2 = config['seed1'],config['seed2']

    total_data = dataset(data,Type,N_size,input_size).data
    X_predict = np.array(total_data)
    
    if total_data.size == 0 :
        flag += 1
        continue
        
    results_df = pd.DataFrame()
    kf = KFold(n_splits=k ,shuffle=True,random_state=seed1)
    
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        model = get_models(config['model'],input_size,epsilon)
        if flag == 0 or flag == 3:
            total_data2 = dataset(data2,Type,N_size,input_size)
            total_data2 = np.array(total_data2,dtype = object)
            X = total_data2[:,0]
            y = total_data2[:,1].astype(np.float32)

            same_seeds(seed2)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            scaler = MinMaxScaler()
            X_train,X_test,y_train,y_test,scaler = data_deal(X_train,X_test,y_train,y_test,scaler,input_size,flag)
            optimizer = optim.Adam(model.parameters(),lr=lr)
            scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda x: 1 - x / 100, last_epoch=-1)
            X_predict = scaler.transform(total_data)
            X_predict = torch.tensor(X_predict, dtype=torch.float32)

            model.load_state_dict(torch.load(f'{para_path}/model_{flag + 1}_fold_{fold + 1}.ckpt'))
            model.eval()
            with torch.no_grad():
                _, test_outputs = model(X_predict)
                y_pred = test_outputs.to('cpu')
                y_pred = np.array(y_pred)
                X_predict = scaler.inverse_transform(X_predict)
            
        else :
            filename = f'{para_path}/model_{flag + 1}_fold_{fold + 1}.pkl'
            model = joblib.load(filename)
            X_predict = np.concatenate(X_predict).reshape(len(X_predict),input_size)
            y_pred = model.predict(X_predict)
        
        if fold ==0:
            results_df = pd.DataFrame(X_predict, columns=[str(i) for i in range(1, input_size + 1)])
        results_df[f'Predict_{fold+1}'] = y_pred
    
    results_df.to_excel(f'test_data_and_predict_{flag + 1}.xlsx', index=False)
    
    flag += 1

In [120]:
print('Running End')

Running End
